In [1]:
import numpy as np 
import pandas as pd 


In [2]:
disease = pd.read_csv('heart.csv')
disease.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [3]:
#Split the data into train and test set
from sklearn.model_selection import train_test_split

train, test = train_test_split(disease, test_size = 0.2, random_state = 1)

In [4]:
X_train = train.drop(['target','fbs'],axis=1)
y_train = train['target']

X_test = train.drop(['target','fbs'],axis=1)
y_test = train['target']

In [5]:
from hyperdash import Experiment

In [10]:
from sklearn.linear_model import SGDClassifier

#Declare your Experiment object each run
exp = Experiment('SGDClassifier')

#Record the value of the hyperparameter alpha
alpha = exp.param('alpha', 0.01)

sgd_model = SGDClassifier(random_state=1, alpha = alpha)
sgd_model.fit(X_train, y_train)


{ alpha: 0.01 }


SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=1, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

[{"varName": "SGDClassifier", "varType": "ABCMeta", "varSize": "2008", "varShape": "", "varContent": "<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>"}, {"varName": "X_test", "varType": "DataFrame", "varSize": "25168", "varShape": "(242, 12)", "varContent": "Column names: age, sex, cp, trestbps, chol, restecg, thalach, exang, oldpeak, slope, ca, thal"}, {"varName": "X_train", "varType": "DataFrame", "varSize": "25168", "varShape": "(242, 12)", "varContent": "Column names: age, sex, cp, trestbps, chol, restecg, thalach, exang, oldpeak, slope, ca, thal"}, {"varName": "alpha", "varType": "float", "varSize": "24", "varShape": "", "varContent": "0.01"}, {"varName": "disease", "varType": "DataFrame", "varSize": "34064", "varShape": "(303, 14)", "varContent": "Column names: age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal, target"}, {"varName": "exp", "varType": "Experiment", "varSize": "64", "varShape": "", "varContent": "<hyperdash.exper

limit_output extension: Maximum message size of 10000 exceeded with 10571 characters

In [11]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(sgd_model, X_train, y_train).mean()
exp.metric('Accuracy', score)

Evaluate the perfomance using Confusion Matrix

In [12]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

sgd_predictions = cross_val_predict(sgd_model, X_train, y_train)
tn, fp, fn, tp  = confusion_matrix(y_train, sgd_predictions).ravel()

exp.metric('True Negative', tn)
exp.metric('False Positive', fp)
exp.metric('False Negative', fn)
exp.metric('True Positive', tp) 




In [13]:
exp.end()

In [73]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from hyperdash import monitor

@monitor('RandomForestClassifier')
def random_forest(exp):
    
    n_estimators = 100
    forest_clf = RandomForestClassifier(random_state=1, n_estimators= n_estimators)
    forest_predictions = cross_val_predict(forest_clf, X_train, y_train )
    f1 = metrics.f1_score
    exp.metric('f1', f1)